## Recasting

- This time, with and without prompting 
- Instructions: 
    - Please answer the following yes-no question about this sentence: <Sentence>
- Volition: 
    - ``In the event "<event>", did the <arg> act on purpose?``
    - Need to edit events and args to make grammatical 
- Change of state
    - ``In the event "<event>", did the state of <arg> change?``






In [2]:
from decomp import UDSCorpus 


c = UDSCorpus(split='dev')

In [26]:
import sys 
from tqdm import tqdm 
volition_examples = []
change_of_state_examples = []


def extract_prompt_info(graph, edge): 
    pred_node, arg_node = edge 
    try:
        pred = graph.head(pred_node)[1][0]
        arg = graph.head(arg_node)[1][0]
    except IndexError:
        pred, arg = None, None
    return pred, arg

for gname in tqdm(c): 
    g = c[gname]
    sent = g.sentence
    try:
        subg = g.semantics_subgraph
    except AttributeError:
        continue
    if len(subg.nodes) == 0:
        continue
    for edge in subg.edges: 
        n1, n2 = edge 

        edge_props = subg.edges[edge]
        if "protoroles" in edge_props: 
            if "volition" in edge_props['protoroles'] and abs(edge_props['protoroles']['volition']['value']) > 1: 
                pred, arg = extract_prompt_info(g, edge)
                volition_example = (gname, edge, pred, arg, sent, edge_props['protoroles']['volition'])
                volition_examples.append(volition_example)
                
            if "change_of_state" in edge_props['protoroles'] and abs(edge_props['protoroles']['change_of_state']['value']) > 1: 
                pred, arg = extract_prompt_info(g, edge)
                change_of_state_example = (gname, edge, pred, arg, sent, edge_props['protoroles']['change_of_state'])
                change_of_state_examples.append(change_of_state_example)


100%|██████████| 2002/2002 [00:00<00:00, 6119.07it/s]


In [30]:

class VolitionTemplate:
    def __init__(self, sent, pred, arg, value=None): 
        self.sent = sent 
        self.pred = pred    
        self.arg = arg
        self.value = value 

    def __str__(self):
        to_ret = f"Sentence: \"{self.sent}\"\n" + \
                 f"In the event \"{self.pred}\", does the participant \"{self.arg}\" act with volition?" 
        if self.value is not None:
            if self.value > 0: 
                ans = "Yes" 
            else:
                ans = "No"
            to_ret += f"\n{ans}" 
        return to_ret 

class ChangeOfStateTemplate:
    def __init__(self, sent, pred, arg, value=None): 
        self.sent = sent 
        self.pred = pred    
        self.arg = arg
        self.value = value

    def __str__(self):
        to_ret = f"Sentence: \"{self.sent}\"\n" + \
                   "In the event \"{self.pred}\", does the state of the participant \"{self.arg}\" change?" 
        if self.value is not None: 
            if self.value > 0: 
                ans = "Yes" 
            else:
                ans = "No"
            to_ret += f"\n{ans}" 
        return to_ret 

In [48]:
import re

# limit sentences to 35 tokens to avoid overly complicated ones 
max_len = 35

volition_templates = []
change_of_state_templates = []
for example in volition_examples:
    gname, edge, pred, arg, sent, val_dict = example
    if len(re.split("\s+", sent)) > max_len:
        continue
    t = VolitionTemplate(sent, pred, arg, val_dict['value'])
    volition_templates.append(t)

for example in change_of_state_examples:
    gname, edge, pred, arg, sent, val_dict = example
    if len(re.split("\s+", sent)) > max_len:
        continue
    t = ChangeOfStateTemplate(sent, pred, arg, val_dict['value'])
    change_of_state_templates.append(t)

In [34]:
import numpy as np

np.random.seed(12)

np.random.shuffle(volition_templates)
np.random.shuffle(change_of_state_templates)

# subset to balance yes and no 
yes_volition_templates = [x for x in volition_templates if x.value > 0]
no_volition_templates = [x for x in volition_templates if x.value < 0]

yes_cos_templates = [x for x in change_of_state_templates if x.value > 0]
no_cos_templates = [x for x in change_of_state_templates if x.value < 0]

min_volition = min(len(yes_volition_templates), len(no_volition_templates))
min_cos = min(len(yes_cos_templates), len(no_cos_templates))

samp_yes_volition_templates = np.random.choice(yes_volition_templates, size=min_volition, replace=False).tolist()
samp_no_volition_templates = np.random.choice(no_volition_templates, size=min_volition, replace=False).tolist()
samp_yes_cos_templates = np.random.choice(yes_cos_templates, size=min_cos, replace=False).tolist()
samp_no_cos_templates = np.random.choice(no_cos_templates, size=min_cos, replace=False).tolist() 


balanced_volition_templates = samp_yes_volition_templates + samp_no_volition_templates
balanced_cos_templates = samp_yes_cos_templates + samp_no_cos_templates

In [50]:
instr_str = "Answer this yes-no question about the following sentence.\n"
def make_prompts(templates, num_prompts, prefix_size = 3, instructions = instr_str):
    np.random.shuffle(templates)
    prompts = []
    max_num_prompts = min(num_prompts, int(len(templates)/(prefix_size + 1)))
    for i in range(max_num_prompts): 
        template_idxs = [i for i in range(len(templates))]
        chosen_idxs = np.random.choice(template_idxs, size=prefix_size + 1, replace=False).tolist()
        chosen = [templates[i] for i in chosen_idxs]
        templates = [x for i, x in enumerate(templates) if i not in chosen_idxs]
        # set last val to None so that it doesn't show 
        chosen[-1].value = None
        prompt = instructions + "\n".join([str(t) for t in chosen])
        prompts.append(prompt)
    return prompts 

In [51]:
import json 
for s in range(0, 4): 
    volition_prompts = make_prompts(balanced_volition_templates, 40, prefix_size=s) 
    cos_prompts = make_prompts(balanced_cos_templates, 40, prefix_size=s) 

    with open(f"../../data/agent_patient/volition_prefix_{s}.json") as f1:
        json.dump(volition_prompts, f1)
    with open(f"../../data/agent_patient/change_of_state_prefix_{s}.json") as f1:
        json.dump(cos_prompts, f1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/agent_patient/volition_prefix_0.json'